<h3>准备</h3>

<h4>导入包</h4>

In [1]:
import os
import json
import requests
from time import localtime, strftime
from typing import Optional, Union
from PIL.Image import Image, open, fromarray
from PIL.ImageFilter import FIND_EDGES
import numpy as np
import tensorflow as tf
from tensorflow import Tensor
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, LeakyReLU, Flatten, Lambda
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Rescaling
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import History, Callback, TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.backend import clear_session
from sklearn.metrics import classification_report

<h4>设置tensorflow</h4>

In [2]:
tf.compat.v1.enable_eager_execution()
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

<h3>准备数据</h3>

In [3]:
train_dir = os.path.join("../data/pic/", "train")
test_dir = os.path.join("../data/pic/", "test")

<h4>边缘检测</h4>

In [4]:
def find_edges(img: Union[np.ndarray, Image]) -> np.ndarray:
    """
    图片边缘检测\n
    :param img: 一张图片
    :return: 边缘检测后的图片
    """
    if isinstance(img, np.ndarray):
        image: Image = fromarray(img.astype(np.uint8))
    else:
        image: Image = img
    return np.array(image.filter(FIND_EDGES)).astype("float")

<h4>数据预处理</h4>

In [9]:
train_gen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=360,
    width_shift_range=0.03,
    height_shift_range=0.03,
    fill_mode='nearest',
    validation_split=0.2
)
val_gen = ImageDataGenerator(
    validation_split=0.2
)

In [10]:
train_ds = train_gen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    seed=123,
    subset="training"
)
val_ds = train_gen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    seed=123,
    subset="validation"
)

Found 89 images belonging to 2 classes.
Found 21 images belonging to 2 classes.


<h3>搭建模型并训练</h3>

<h4>搭建模型</h4>

In [11]:
clear_session()
model = Sequential([
    Resizing(160, 160, name="resize", input_shape=(160, 160, 3)),
    Rescaling(1.0 / 255, name="rescale"),
    Conv2D(filters=32, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d1"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_1"),
    Conv2D(filters=64, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d2"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_2"),
    Conv2D(filters=128, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d3"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_3"),
    Conv2D(filters=32, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d4"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_4"),
    Conv2D(filters=16, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d5"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_5"),
    Flatten(data_format="channels_last", name="flatten_6"),
    Dense(units=16, activation="relu", name="dense_6"),
    Dense(units=2, activation="sigmoid", name="dense_7")
], name="pic_clf")
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=1e-4),
    metrics=["accuracy"]
)
model.summary()

Model: "pic_clf"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resize (Resizing)            (None, 160, 160, 3)       0         
_________________________________________________________________
rescale (Rescaling)          (None, 160, 160, 3)       0         
_________________________________________________________________
conv2d1 (Conv2D)             (None, 158, 158, 32)      896       
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d2 (Conv2D)             (None, 77, 77, 64)        18496     
_________________________________________________________________
maxpool2d_2 (MaxPooling2D)   (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d3 (Conv2D)             (None, 36, 36, 128)       7385

<h4>模型训练</h4>

In [12]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=120,
    batch_size=32,
    verbose=1,
    workers=-1,
    use_multiprocessing=True,
    callbacks=[
        TensorBoard(log_dir="../logs/" + strftime("%Y%m%d-%H%M%S", localtime()))
    ]
)

Epoch 1/120
1/3 [=========>....................] - ETA: 0s - loss: 0.6993 - accuracy: 0.1200WARNING:tensorflow:From c:\users\drzon\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
3/3 [==============================] - 2s 542ms/step - loss: 0.6956 - accuracy: 0.2921 - val_loss: 0.6919 - val_accuracy: 0.7143
Epoch 2/120
3/3 [==============================] - 1s 185ms/step - loss: 0.6902 - accuracy: 0.7191 - val_loss: 0.6868 - val_accuracy: 0.7143
Epoch 3/120
3/3 [==============================] - 1s 189ms/step - loss: 0.6844 - accuracy: 0.7191 - val_loss: 0.6815 - val_accuracy: 0.7143
Epoch 4/120
3/3 [==============================] - 1s 174ms/step - loss: 0.6796 - accuracy: 0.7191 - val_loss: 0.6765 - val_accuracy: 0.7143
Epoch 5/120
3/3 [============================

3/3 [==============================] - 1s 187ms/step - loss: 0.5542 - accuracy: 0.7191 - val_loss: 0.5495 - val_accuracy: 0.7143
Epoch 57/120
3/3 [==============================] - 1s 192ms/step - loss: 0.5471 - accuracy: 0.7191 - val_loss: 0.5463 - val_accuracy: 0.7143
Epoch 58/120
3/3 [==============================] - 1s 180ms/step - loss: 0.5454 - accuracy: 0.7191 - val_loss: 0.5341 - val_accuracy: 0.7143
Epoch 59/120
3/3 [==============================] - 1s 178ms/step - loss: 0.5390 - accuracy: 0.7191 - val_loss: 0.5300 - val_accuracy: 0.7143
Epoch 60/120
3/3 [==============================] - 1s 187ms/step - loss: 0.5342 - accuracy: 0.7191 - val_loss: 0.5249 - val_accuracy: 0.7143
Epoch 61/120
3/3 [==============================] - 1s 170ms/step - loss: 0.5279 - accuracy: 0.7191 - val_loss: 0.5191 - val_accuracy: 0.7143
Epoch 62/120
3/3 [==============================] - 1s 186ms/step - loss: 0.5173 - accuracy: 0.7191 - val_loss: 0.5053 - val_accuracy: 0.7143
Epoch 63/120
3/3 [=

Epoch 114/120
3/3 [==============================] - 1s 172ms/step - loss: 0.3151 - accuracy: 1.0000 - val_loss: 0.3145 - val_accuracy: 1.0000
Epoch 115/120
3/3 [==============================] - 1s 187ms/step - loss: 0.3156 - accuracy: 1.0000 - val_loss: 0.3136 - val_accuracy: 1.0000
Epoch 116/120
3/3 [==============================] - 1s 170ms/step - loss: 0.3139 - accuracy: 1.0000 - val_loss: 0.3137 - val_accuracy: 1.0000
Epoch 117/120
3/3 [==============================] - 1s 170ms/step - loss: 0.3146 - accuracy: 1.0000 - val_loss: 0.3136 - val_accuracy: 1.0000
Epoch 118/120
3/3 [==============================] - 1s 177ms/step - loss: 0.3143 - accuracy: 1.0000 - val_loss: 0.3135 - val_accuracy: 1.0000
Epoch 119/120
3/3 [==============================] - 1s 190ms/step - loss: 0.3145 - accuracy: 1.0000 - val_loss: 0.3137 - val_accuracy: 1.0000
Epoch 120/120
3/3 [==============================] - 1s 186ms/step - loss: 0.3139 - accuracy: 1.0000 - val_loss: 0.3139 - val_accuracy: 1.0000

In [ ]:
tensorboard --logdir ../logs

In [40]:
save_model(model, "home/centos/pic_clf/4")      # save_model(model, "../models/pic_clf/4")

INFO:tensorflow:Assets written to: ../models/pic_clf/1\assets


<h3>部署模型并预测</h3>

<h4>部署模型</h4>

<h5>安装docker</h5>

In [ ]:
!sudo yum install -y epel-release
!sudo yum install https://get.docker.com/rpm/1.7.1/centos-6/RPMS/x86_64/docker-engine-1.7.1-1.el6.x86_64.rpm
!docker version
!sudo service docker start

<h5>拉取镜像</h5>

In [ ]:
!docker pull tensorflow/serving:2.3.0

<h5>部署服务</h5>

In [ ]:
!docker run -t \
    -p 8501:8501 \
    -v "/home/centos/pic_clf:/models/pic_clf" \
    -e MODEL_NAME=pic_clf \
    tensorflow/serving:2.3.0

<h4>测试数据</h4>

<h5>读取测试数据</h5>

In [18]:
x_test, y_test = list(image_dataset_from_directory(test_dir, image_size=(160, 160)))[1]

Found 63 files belonging to 2 classes.


In [19]:
x_test = x_test.numpy()
y_test = y_test.numpy()

<h5>预测数据</h5>

In [20]:
data = json.dumps({"instances": x_test.tolist()})
headers = {"content-type": "application/json"}
response = requests.post(
    url="http://81.70.8.71:8501/v1/models/pic_clf:predict",
    data=data,
    headers=headers
)
if response.status_code != 200:
    raise ValueError(response.text)

In [21]:
y_pred_prob = np.array(json.loads(response.text)["predictions"])

In [23]:
y_pred = np.where(y_pred_prob[:, 0] > y_pred_prob[:, 1], 0, 1).ravel()

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        20

    accuracy                           1.00        31
   macro avg       1.00      1.00      1.00        31
weighted avg       1.00      1.00      1.00        31

